In [1]:
import pandas as pd
import os
from lib.preprocess import preprocessor

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def process_df(df):
    df = df[df.content.notna()].reset_index()
    df['content'] = df.content.map(preprocessor)
    df = df[df.content.notna()].reset_index()
    df['repo_owner'] = df.repo_name.map(lambda s: s.split('/')[0])
    return df

In [4]:
path = './oss_bigquery_exports/readmes'
files = list(os.walk(path))[0][2]
dfs = (pd.read_csv('{}/{}'.format(path, f)) for f in files)

In [5]:
with open('file_lookup.csv', 'w') as f:
    for df in dfs:
        df = process_df(df)
        for i,c in zip(df.id, df.content):
            f.write('{},{}\n'.format(i,c))

### Treat owner like collaborative filtering? 

In [101]:
by_owner = df.groupby('repo_owner').apply(lambda df: ' \t '.join(df.content))

In [102]:
with open('by_owner.txt', 'w') as f:
    for c in by_owner:
        f.write(c + '\n')